# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2b7b0d11ac-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("Place of Birth, Death, and Burial") 

Consider the following exploratory information need:

> You want to visit cities connected to famous writers and poets, and you are deciding wether to visit France or Germany

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q183`     | Germany       | node |
| `wd:Q142`     | France        | node |
| `wd:Q90`      | Paris         | node |
| `wd:Q49757`   | Poet          | node |
| `wd:Q36180`   | Writer        | node |
| `wd:Q501`     | Charles Baudelaire  | node      |
| `wd:Q272208`  | Montparnasse Cemetery       | node |


Also consider

```
?p wdt:P27 wd:Q142  . 
?p wdt:P106 wd:Q36180  . 
```

is the BGP to retrieve all **French writers**

## Workload Goals

1. Identify the BGP that connect people to their place of birth or place, death, or burial

2. Identify the BGP to obtain the country in which a place is located

3. How many poets and writers  have a place of birth, death, or burial in Germany and France?

4. Analyze cities across the two countries
 
   4.1 Is there any poet for which the birth place and the place of burial are located in the same city either in Germany or France?
   
   4.2 Which cities host the place of birth of the larger number of poets or writers across the two countries?
   
   4.3 What are the top 3 cities in each country that you could visit? Based on what criteria?


In [3]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P27 wd:Q142  . 
?p wdt:P106 wd:Q36180  . 
} 
GROUP BY ?cult  ?arch
"""

print("Predicates")
run_query(queryString)

Predicates
[('callret-0', '14400')]


1

<h3>TASK 1 STARTS HERE </h3> 
Identify the BGP that connect people to their place of birth or place, death, or burial

Search properties of french writers.

In [5]:
#Q1
#search properties of french writers
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?f wdt:P27 wd:Q142;
   wdt:P106 wd:Q36180;
   ?p ?o.
   
#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3363'), ('pname', 'Tennis Hall of Fame player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4944'), ('pname', 'Premiers préfets ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pname', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pname', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1045'), ('pname', 'Sycomore ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('pname', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.or

25

Retrieve "Place of burial" of french writers using P119 found above.

In [6]:
#Q2
#use P119 (place of burial) found above
queryString = """
SELECT DISTINCT ?fname ?o ?oname
WHERE { 

?f wdt:P27 wd:Q142;
   wdt:P106 wd:Q36180;
   wdt:P119 ?o.
   
#Labels
?f <http://schema.org/name> ?fname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('fname', 'Louise Colet'), ('o', 'http://www.wikidata.org/entity/Q317271'), ('oname', 'Verneuil-sur-Avre')]
[('fname', 'Zénaïde Fleuriot'), ('o', 'http://www.wikidata.org/entity/Q12642'), ('oname', 'Morbihan')]
[('fname', 'Pierre Delanoë'), ('o', 'http://www.wikidata.org/entity/Q8532961'), ('oname', 'Cimetière de Fourqueux')]
[('fname', 'Edmonde Charles-Roux'), ('o', 'http://www.wikidata.org/entity/Q2972480'), ('oname', 'Saint Pierre Cemetery')]
[('fname', 'Raymond Queneau'), ('o', 'http://www.wikidata.org/entity/Q752255'), ('oname', 'Juvisy-sur-Orge')]
[('fname', 'Julie Manet'), ('o', 'http://www.wikidata.org/entity/Q1092107'), ('oname', 'Passy Cemetery')]
[('fname', 'Abbot Suger'), ('o', 'http://www.wikidata.org/entity/Q184407'), ('oname', 'Basilica of Saint-Denis')]
[('fname', 'Édouard Vaillant'), ('o', 'http://www.wikidata.org/entity/Q208538'), ('oname', 'Vierzon')]
[('fname', 'Adolphe Willette'), ('o', 'http://www.wikidata.org/entity/Q272208'), ('oname', 'Montparnasse Cem

25

Retrieve some properties related to cemetries.

In [7]:
#Q3
#closer look at entities 
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?f wdt:P27 wd:Q142;
   wdt:P106 wd:Q36180;
   wdt:P119 ?c.

?c ?p ?o .
   
#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P10'), ('pname', 'video')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1281'), ('pname', 'WOEID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pname', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P1376'), ('pname', 'capital of')]
[('p', 'http://www.w

25

Try to use P1464 and check where it can lead us.

In [8]:
#Q4
#properties P1464 P1465 seem to lead to the reverse path we want to find
queryString = """
SELECT DISTINCT ?c ?cname ?o ?oname
WHERE { 

?f wdt:P27 wd:Q142;
   wdt:P106 wd:Q36180;
   wdt:P119 ?c.

?c wdt:P1464 ?o .
   
#Labels
?c <http://schema.org/name> ?cname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q12642'), ('cname', 'Morbihan'), ('o', 'http://www.wikidata.org/entity/Q8081092'), ('oname', 'Category:Births in Morbihan')]
[('c', 'http://www.wikidata.org/entity/Q55'), ('cname', 'Netherlands'), ('o', 'http://www.wikidata.org/entity/Q7463294'), ('oname', 'Category:Births in the Netherlands')]
[('c', 'http://www.wikidata.org/entity/Q12543'), ('cname', 'Hauts-de-Seine'), ('o', 'http://www.wikidata.org/entity/Q9229576'), ('oname', 'Category:Births in Hauts-de-Seine')]
[('c', 'http://www.wikidata.org/entity/Q1854'), ('cname', 'Ho Chi Minh City'), ('o', 'http://www.wikidata.org/entity/Q18523866'), ('oname', 'Category:Births in Ho Chi Minh City')]
[('c', 'http://www.wikidata.org/entity/Q365'), ('cname', 'Cologne'), ('o', 'http://www.wikidata.org/entity/Q11078895'), ('oname', 'Category: Born in Cologne')]
[('c', 'http://www.wikidata.org/entity/Q456327'), ('cname', "Ville-d'Avray"), ('o', 'http://www.wikidata.org/entity/Q9228011'), ('oname', "Ca

25

Retrieve the properties of the triples where "born in Paris" is the object.

In [10]:
#Q5
#check where reverse path lead
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?s ?p wd:Q9226347. #born in Paris
   
#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('pname', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pname', 'opposite of')]


2

As above but using "Born in Paris" as subject.

In [11]:
#Q6
#check where reverse path lead
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

wd:Q9226347 ?p ?o. #born in Paris
   
#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4224'), ('pname', 'category contains')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pname', 'opposite of')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]


5

Retrieve the proprties of writers that are related to a city. I add P137 (capital of) to exclude place that are not city.

In [12]:
#Q7
#that properties don't look useful
#search properties of writers, add P1376 (capital of) found in Q3 to have only cities as objects
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?f wdt:P106 wd:Q36180;  #writers
   ?p ?city.

?city wdt:P1376 ?o. #capital of
   
#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pname', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pname', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('pname', 'residence')]
[('p', 'http://www.wikidata.org/prop/direct/P937'), ('pname', 'work location')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P66'), ('pname', 'ancestral home')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pname', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P7153'), ('pname', 'significant place')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at')]
[('p', 'http://www.

25

Above I found P19 (place of birth) and P20 (place of death). Retrieve some french writers and their place of birth.

In [13]:
#Q8
#use P19 and P20 found above
queryString = """
SELECT DISTINCT ?f ?fname ?o ?oname
WHERE { 

?f  wdt:P27 wd:Q142;
    wdt:P106 wd:Q36180;  #writers
    wdt:P19 ?o.
   
#Labels
?f <http://schema.org/name> ?fname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('f', 'http://www.wikidata.org/entity/Q21775921'), ('fname', 'Ferdinand Pouy'), ('o', 'http://www.wikidata.org/entity/Q1373216'), ('oname', 'Villiers-sur-Tholon')]
[('f', 'http://www.wikidata.org/entity/Q63485595'), ('fname', 'Mariette Moreaud Monnet'), ('o', 'http://www.wikidata.org/entity/Q82185'), ('oname', 'La Rochelle')]
[('f', 'http://www.wikidata.org/entity/Q16186086'), ('fname', 'Jacob Neyts'), ('o', 'http://www.wikidata.org/entity/Q12994'), ('oname', 'Bruges')]
[('f', 'http://www.wikidata.org/entity/Q16196357'), ('fname', 'Anacharsis Combes'), ('o', 'http://www.wikidata.org/entity/Q188776'), ('oname', 'Castres')]
[('f', 'http://www.wikidata.org/entity/Q17429277'), ('fname', 'Gabrielle Réval'), ('o', 'http://www.wikidata.org/entity/Q13395'), ('oname', 'Viterbo')]
[('f', 'http://www.wikidata.org/entity/Q3091464'), ('fname', 'Xavier Forneret'), ('o', 'http://www.wikidata.org/entity/Q16121'), ('oname', 'Beaune')]
[('f', 'http://www.wikidata.org/entity/Q674280'), ('fname',

25

Retrieve place of birth, death and burial of Baudelaire.

In [14]:
#Q9
#retrieve place of birth, death and burial of Baudelaire
queryString = """
SELECT DISTINCT ?bname ?dname ?buname
WHERE { 

wd:Q501 wdt:P19 ?birth;
        wdt:P20 ?death;
        wdt:P119 ?burial.

   
#Labels
?birth <http://schema.org/name> ?bname .
?death <http://schema.org/name> ?dname .
?burial <http://schema.org/name> ?buname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('bname', 'Paris'), ('dname', 'Paris'), ('buname', 'Montparnasse Cemetery')]


1

<h3> TASK 1 ENDS HERE </h3>
In the code below there is the BGP to obtain someone's place of birth, death and burial.

In [15]:
#TASK 1 ENDS HERE:
#The BGP to obtain someone's place of birth, death and burial is the following:
queryString = """
SELECT ?s ?birth ?death ?burial
WHERE { 

?s      wdt:P19 ?birth;
        wdt:P20 ?death;
        wdt:P119 ?burial.
} 
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q310715'), ('birth', 'http://www.wikidata.org/entity/Q1289'), ('death', 'http://www.wikidata.org/entity/Q1095443'), ('burial', 'http://www.wikidata.org/entity/Q272208')]
[('s', 'http://www.wikidata.org/entity/Q4093892'), ('birth', 'http://www.wikidata.org/entity/Q45003206'), ('death', 'http://www.wikidata.org/entity/Q45003206'), ('burial', 'http://www.wikidata.org/entity/Q45003206')]
[('s', 'http://www.wikidata.org/entity/Q456991'), ('birth', 'http://www.wikidata.org/entity/Q956524'), ('death', 'http://www.wikidata.org/entity/Q487096'), ('burial', 'http://www.wikidata.org/entity/Q126453')]
[('s', 'http://www.wikidata.org/entity/Q553172'), ('birth', 'http://www.wikidata.org/entity/Q385070'), ('death', 'http://www.wikidata.org/entity/Q8818'), ('burial', 'http://www.wikidata.org/entity/Q3682887')]
[('s', 'http://www.wikidata.org/entity/Q710504'), ('birth', 'http://www.wikidata.org/entity/Q180139'), ('death', 'http://www.wikidata.org/entity/Q1

5

<h3> TASK 2 STARTS HERE </h3>
Identify the BGP to obtain the country in which a place is located.

Retrieve some properties of Montparnasse cemetry.

In [3]:
#Q10
#analyze wd:Q272208
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

wd:Q272208 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1329'), ('pname', 'phone number')]
[('p', 'http://www.wikidata.org/prop/direct/P1566'), ('pname', 'GeoNames ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1766'), ('pname', 'place name sign')]
[('p', 'http://www.wikidata.org/prop/direct/P1791'), ('pname', 'category of people buried here')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2025'), ('pname', 'Find A Grave cemetery ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2046'), ('pname', 'area')]
[('p', 'http://www.wikidata.org/prop/direct/P281'), ('pname', 'postal code')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3182'), ('pname', 'FANTOIR co

25

In the previous result set, I found out that P17 is the property for country. Test it on Montparnasse cemetry.

In [4]:
#Q11
#analyze wd:Q272208 using P17 country
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

wd:Q272208 wdt:P17 ?o.

#Labels
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q142'), ('oname', 'France')]


1

In Q10 I also found P131 (located in the administrative territorial entity) and P495 (country of origin). Check the results if using Montparnasse as
subject.

In [5]:
#Q12
#check also P131, P495 found in Q10
queryString = """
SELECT DISTINCT ?area ?aname ?origin ?oname
WHERE { 

wd:Q272208 wdt:P131 ?area;
           wdt:P495 ?origin.

#Labels
?area <http://schema.org/name> ?aname .
?origin <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('area', 'http://www.wikidata.org/entity/Q187153'), ('aname', '14th arrondissement of Paris'), ('origin', 'http://www.wikidata.org/entity/Q142'), ('oname', 'France')]


1

In [6]:
#Q13
#analyze wd:Q272208 using P31
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

wd:Q272208 wdt:P31 ?o.

#Labels
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2972684'), ('oname', 'Parisian cemetery')]


1

In [7]:
#Q14
#iterate over P31
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

wd:Q272208 wdt:P31/wdt:P31 ?o.

#Labels
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
Empty


0

I'd like to see if there are some categories related to historical places/monuments or to distinguish points of interest in general. Retrieve the 
properties of "Parisian cemetry" found in Q13.

In [8]:
#Q15
#looking for something related to place/monument
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE { 

wd:Q2972684 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8959882'), ('oname', 'Category:Cemeteries in Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q142'), ('oname', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P126'), ('pname', 'maintained by'), ('o', 'http://www.wikidata.org/entity/Q90'), ('oname', 'Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q39614'), ('oname', 'cemetery')]


4

As previous query but using "cemetry" (Q39614).

In [9]:
#Q16
#looking for something related to place/monument using Q39614 found above
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE { 

wd:Q39614 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2046'), ('oname', 'area')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1705'), ('oname', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2324'), ('oname', 'quantity buried')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q173387'), ('oname', 'grave')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1497375'), ('oname', 'architectural ensemble')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2351'), ('oname', 'number of graves')]
[('p', 'http://www.w

25

Retrieve just the properties.

In [11]:
#Q17
#looking for something related to place/monument using Q39614 found above
#as before but only properties
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {

wd:Q39614 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pname', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pname', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pname', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pname', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('pname', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1557'), ('pname', 'manifestation of')]

50

Use property P1766 (place name sign) found in Q10

In [13]:
#Q18
#use property P1766 found in Q10
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {

wd:Q272208 wdt:P1766 ?o.

#Labels
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
Empty


0

Try to build a path using Q1497375 (architecture ensemble) found in Q16.

In [14]:
#Q19
#try to connect with Q1497375 (architecture ensemble) found in Q16
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {

?s wdt:P17 wd:Q142; #france
   wdt:P31 ?o.
?o wdt:P279 wd:Q1497375.

#Labels
?s <http://schema.org/name> ?sname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q56685193'), ('sname', 'Collège de Pontlevoy')]
[('s', 'http://www.wikidata.org/entity/Q66715783'), ('sname', 'Habitation Gradis')]
[('s', 'http://www.wikidata.org/entity/Q22925567'), ('sname', 'Domaine des Granges à Hénon')]
[('s', 'http://www.wikidata.org/entity/Q22979508'), ('sname', 'Domaine du château de Saint-Symphorien-des-Monts')]
[('s', 'http://www.wikidata.org/entity/Q5289861'), ('sname', 'Domaine de Terre Blanche')]
[('s', 'http://www.wikidata.org/entity/Q856882'), ('sname', 'Bicêtre')]
[('s', 'http://www.wikidata.org/entity/Q3034410'), ('sname', 'Domaine de Beauregard')]
[('s', 'http://www.wikidata.org/entity/Q22996538'), ('sname', 'Domaine de Vallète')]
[('s', 'http://www.wikidata.org/entity/Q88413287'), ('sname', 'Domaine Adolphe-Chérioux')]
[('s', 'http://www.wikidata.org/entity/Q22954710'), ('sname', 'Domaine de la Forêt')]
[('s', 'http://www.wikidata.org/entity/Q107470992'), ('sname', 'mansion of Madame Élisabeth')]
[('s',

25

In [15]:
#Q20
#iterate over P31
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {

?s wdt:P17 wd:Q142; #france
   wdt:P31* ?o.
?o wdt:P279 wd:Q1497375.

#Labels
?s <http://schema.org/name> ?sname .


} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q66715783'), ('sname', 'Habitation Gradis')]
[('s', 'http://www.wikidata.org/entity/Q22925567'), ('sname', 'Domaine des Granges à Hénon')]
[('s', 'http://www.wikidata.org/entity/Q22979508'), ('sname', 'Domaine du château de Saint-Symphorien-des-Monts')]
[('s', 'http://www.wikidata.org/entity/Q5289861'), ('sname', 'Domaine de Terre Blanche')]
[('s', 'http://www.wikidata.org/entity/Q856882'), ('sname', 'Bicêtre')]
[('s', 'http://www.wikidata.org/entity/Q3034410'), ('sname', 'Domaine de Beauregard')]
[('s', 'http://www.wikidata.org/entity/Q22996538'), ('sname', 'Domaine de Vallète')]
[('s', 'http://www.wikidata.org/entity/Q88413287'), ('sname', 'Domaine Adolphe-Chérioux')]
[('s', 'http://www.wikidata.org/entity/Q22954710'), ('sname', 'Domaine de la Forêt')]
[('s', 'http://www.wikidata.org/entity/Q107470992'), ('sname', 'mansion of Madame Élisabeth')]
[('s', 'http://www.wikidata.org/entity/Q3034418'), ('sname', 'domain of Chantilly')]
[('s', '

25

Retrieve the properties of entity wd:Q66715783 (Habitation Gradis).

In [16]:
#Q21
#closer look to an entity
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {

wd:Q66715783 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pname', 'heritage designation')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P380'), ('pname', 'Mérimée ID')]


9

Try to use art and architeture id property found in Q17.

In [17]:
#Q22
#try to use art and architeture id property found in Q17
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {

?s wdt:P17 wd:Q142; #france
   wdt:P1014 ?o.

#Labels
?s <http://schema.org/name> ?sname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1246307'), ('sname', 'Tornesel')]
[('s', 'http://www.wikidata.org/entity/Q455403'), ('sname', 'grand hotel')]
[('s', 'http://www.wikidata.org/entity/Q174419'), ('sname', 'Second Empire style')]
[('s', 'http://www.wikidata.org/entity/Q129337'), ('sname', 'TGV')]
[('s', 'http://www.wikidata.org/entity/Q56472'), ('sname', 'Shuadit')]
[('s', 'http://www.wikidata.org/entity/Q209086'), ('sname', 'Pantin')]
[('s', 'http://www.wikidata.org/entity/Q42934'), ('sname', 'Cubism')]
[('s', 'http://www.wikidata.org/entity/Q466887'), ('sname', 'Belle Époque')]
[('s', 'http://www.wikidata.org/entity/Q517391'), ('sname', 'Folles')]
[('s', 'http://www.wikidata.org/entity/Q35735'), ('sname', 'Gascon')]
[('s', 'http://www.wikidata.org/entity/Q12107'), ('sname', 'Breton')]
[('s', 'http://www.wikidata.org/entity/Q150'), ('sname', 'French')]
[('s', 'http://www.wikidata.org/entity/Q188'), ('sname', 'German')]
[('s', 'http://www.wikidata.org/entity/Q202216'), ('sn

25

Try to use P1963 found in Q16 (Q811979).

In [19]:
#Q23
#try to use P1963 found in Q16 (Q811979)
#the entity coordinate location could link to place/monument
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {

?s wdt:P17 wd:Q142; #france
   wdt:P1963 ?o.

#Labels
?s <http://schema.org/name> ?sname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q916475'), ('sname', 'Historical Monument')]
[('s', 'http://www.wikidata.org/entity/Q3457689'), ('sname', 'nature reserve in France')]


2

Try to use historical monument entity in order to have a more general definition of monument/point of interest.

In [2]:
#Q24
#try to use historical monument entity
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {

?s ?p wd:Q916475.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pname', 'heritage designation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P4224'), ('pname', 'category contains')]
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('pname', 'significant event')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('pname', 'main subject')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]


9

In [3]:
#Q25
#try to use historical monument entity
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {

wd:Q916475 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pname', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4212'), ('pname', 'PACTOLS thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P6104'), ('pname', 'maintained by WikiProject')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7818'), ('pname', 'French Vikidia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]


12

In [4]:
#Q26
#try to use historical monument entity
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {

?s wdt:P31 wd:Q916475;
   wdt:P17 wd:Q142.

#Labels
?s <http://schema.org/name> ?sname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
Empty


0

Try to use P301 found on Q24.

In [5]:
#Q27
#try to use P301 found on Q24
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {

?s wdt:P301 wd:Q916475.

#Labels
?s <http://schema.org/name> ?sname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q7164760'), ('sname', 'Category:Official historical monuments of France')]


1

In [6]:
#Q28
#wd:Q7164760
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {

wd:Q7164760 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4167836'), ('oname', 'Wikimedia category')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic"), ('o', 'http://www.wikidata.org/entity/Q916475'), ('oname', 'Historical Monument')]


2

Create a graph pattern considering France and historical monument as object. Retrieve some properties. 

In [7]:
#Q29
#use path found in Q20
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {

?s wdt:P17 wd:Q142; #france
   ?p ?o. #some connection
?o wdt:P279 wd:Q916475. #historical monument

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2389'), ('pname', 'organization directed by the office or position'), ('o', 'http://www.wikidata.org/entity/Q35309827'), ('oname', 'Inspection générale des monuments historiques')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10387684'), ('oname', 'monument historique classé')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pname', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q10387575'), ('oname', 'registered historic monument')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pname', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q10387684'), ('oname', 'monument historique classé')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pname', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q10387684'), ('oname', 'monument historique classé')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'h

9

The following BGP is the one to find the historical monuments or places in France.

In [8]:
#Q30
#BGP to find historical france monuments
#use path found in Q20
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {

?s wdt:P17 wd:Q142; #france
   wdt:P1435 ?o. #heritage designation

FILTER ( (?o=wd:Q10387575) || (?o=wd:Q10387684) )

#Labels
?s <http://schema.org/name> ?sname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q334124'), ('sname', 'Remiremont Abbey')]
[('s', 'http://www.wikidata.org/entity/Q15918273'), ('sname', 'Château de Soye')]
[('s', 'http://www.wikidata.org/entity/Q15918342'), ('sname', "Château d'Aynac")]
[('s', 'http://www.wikidata.org/entity/Q16008495'), ('sname', 'Église Saint-Jacques-le-Majeur de Saint-Yaguen')]
[('s', 'http://www.wikidata.org/entity/Q16008496'), ('sname', 'Église Saint-Laurent de Montégut')]
[('s', 'http://www.wikidata.org/entity/Q16008497'), ('sname', 'Église Saint-Lucien de Litz')]
[('s', 'http://www.wikidata.org/entity/Q16008503'), ('sname', 'Église Saint-Hilaire de Salvezou')]
[('s', 'http://www.wikidata.org/entity/Q16008519'), ('sname', "Église Notre-Dame-de-la-Nativité d'Alan")]
[('s', 'http://www.wikidata.org/entity/Q16008524'), ('sname', 'Église Saint-Perdufle de la Masse')]
[('s', 'http://www.wikidata.org/entity/Q16008526'), ('sname', 'Église de Caillac')]
[('s', 'http://www.wikidata.org/entity/Q16008570'), 

25

Retrieve countries of some heritage designation.

In [11]:
#Q31
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {

?s wdt:P17 ?c; 
   wdt:P1435 ?o. #heritage designation

FILTER ( (?o=wd:Q10387575) || (?o=wd:Q10387684) )

#Labels
?c <http://schema.org/name> ?cname .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France')]
[('c', 'http://www.wikidata.org/entity/Q70972'), ('cname', 'Kingdom of France')]
[('c', 'http://www.wikidata.org/entity/Q29'), ('cname', 'Spain')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('cname', 'Italy')]
[('c', 'http://www.wikidata.org/entity/Q252'), ('cname', 'Indonesia')]
[('c', 'http://www.wikidata.org/entity/Q12631'), ('cname', 'Meuse')]
[('c', 'http://www.wikidata.org/entity/Q1137'), ('cname', 'Lorraine')]
[('c', 'http://www.wikidata.org/entity/Q622783'), ('cname', 'Spanish Netherlands')]
[('c', 'http://www.wikidata.org/entity/Q771193'), ('cname', 'Kingdom of Brittany')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('c', 'http://www.wikidata.org/entity/Q298'), ('cname', 'Chile')]
[('c', 'http://www.wikidata.org/entity/Q1009'), ('cname', 'Cameroon')]
[('c', 'http://www.wikidata.org/entity/Q1747689'), ('cname', 'Ancient Rome')]
[('c', 'http://www.wikidata.org/entity/Q403'), 

15

In [12]:
#Q32
#very few countries in the previous result set, get a closer look to heritage designation wd:Q10387575
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {

wd:Q10387575 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q142'), ('oname', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q358'), ('oname', 'heritage site')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q916475'), ('oname', 'Historical Monument')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pname', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P380'), ('oname', 'Mérimée ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q21177718'), ('oname', 'Category:Monuments historiques inscrits')]


5

In [13]:
#Q33
#very few countries in the previous result set, get a closer look to heritage designation wd:Q10387684
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {

wd:Q10387684 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q142'), ('oname', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q358'), ('oname', 'heritage site')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q916475'), ('oname', 'Historical Monument')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pname', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P380'), ('oname', 'Mérimée ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q26938523'), ('oname', 'Category:Monuments historiques classés')]


5

<h3> TASK 2 ENDS HERE </h3>
I found out that P17 is the property that links a entity to its country.
On top of that I tried to explore the graph in order to find a pattern to identify historical monuments or points of interest in a certain country.
As a result I found out the BGP to discover french historical monuments (Q30), I didn't dig more to find a general BGP for historical monuments because I ended up in a dead end and I didn't want to extend the task too much.

<h3> TASK 3 STARTS HERE </h3>
How many poets and writers have a place of birth, death, or burial in Germany and France?

Use the information found in previous task to retrieve the place of birth of poets who are/were born in France.

In [17]:
#Q34
#try to use the information found in the previous task
queryString = """
SELECT ?sname ?birth 
WHERE { 

?s  wdt:P106 wd:Q49757; #poet
    wdt:P19 ?birth.
?birth wdt:P17 wd:Q142.  

#Labels
?s <http://schema.org/name> ?sname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('sname', 'Valery Larbaud'), ('birth', 'http://www.wikidata.org/entity/Q93351')]
[('sname', 'Georges Lafenestre'), ('birth', 'http://www.wikidata.org/entity/Q6548')]
[('sname', 'André Theuriet'), ('birth', 'http://www.wikidata.org/entity/Q656505')]
[('sname', 'Robert de Montesquiou'), ('birth', 'http://www.wikidata.org/entity/Q90')]
[('sname', 'Bonaventure des Périers'), ('birth', 'http://www.wikidata.org/entity/Q16031')]
[('sname', 'Gottlieb Konrad Pfeffel'), ('birth', 'http://www.wikidata.org/entity/Q130994')]
[('sname', 'Dionys Ordinaire'), ('birth', 'http://www.wikidata.org/entity/Q923421')]
[('sname', 'André Bellessort'), ('birth', 'http://www.wikidata.org/entity/Q184154')]
[('sname', 'Guillaume Antoine Le Monnier'), ('birth', 'http://www.wikidata.org/entity/Q668199')]
[('sname', 'Francesc Català i Duran'), ('birth', 'http://www.wikidata.org/entity/Q752284')]


10

Retrieve how many poets are born in France.

In [20]:
#Q35
#try to use the information found in the previous task
#poets born in france
queryString = """
SELECT (COUNT(DISTINCT ?s) AS ?Poets)
WHERE { 

?s  wdt:P106 wd:Q49757; #poet
    wdt:P19 ?birth.
?birth wdt:P17 wd:Q142. 

OPTIONAL{
  ?s wdt:P20 ?death.
  ?death wdt:P17 wd:Q142}
OPTIONAL{
  ?s wdt:P119 ?burial.
  ?burial wdt:P17 wd:Q142}

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('Poets', '3849')]


1

Retrieve poets who are/were born or dead or buried in France.

In [25]:
#Q36
#try to use the information found in the previous task
#poets born,death o buried in france
queryString = """
SELECT (COUNT(DISTINCT ?s) AS ?Poets)
WHERE { 

{?s  wdt:P106 wd:Q49757; #poet
    wdt:P19 ?birth.
?birth wdt:P17 wd:Q142.}

UNION{
  ?s  wdt:P106 wd:Q49757; #poet
      wdt:P20 ?death.
?death wdt:P17 wd:Q142.}

UNION{
  ?s  wdt:P106 wd:Q49757; #poet
      wdt:P119 ?burial.
?burial wdt:P17 wd:Q142.}

} 
"""

print("Results")
run_query(queryString)

Results
[('Poets', '4685')]


1

In [24]:
#Q37
#try to use the information found in the previous task
#poets born,death o buried in france
queryString = """
SELECT (COUNT(DISTINCT ?s) AS ?Born) (COUNT(DISTINCT ?d) AS ?Dead) (COUNT(DISTINCT ?b) AS ?Buried)
WHERE { 

{?s  wdt:P106 wd:Q49757; #poet
    wdt:P19 ?birth.
?birth wdt:P17 wd:Q142.}

UNION{
  ?d  wdt:P106 wd:Q49757; #poet
      wdt:P20 ?death.
?death wdt:P17 wd:Q142.}

UNION{
  ?b  wdt:P106 wd:Q49757; #poet
      wdt:P119 ?burial.
?burial wdt:P17 wd:Q142.}

} 
"""

print("Results")
run_query(queryString)

Results
[('Born', '3849'), ('Dead', '3203'), ('Buried', '710')]


1

Retrieve the poets born in Germany.

In [27]:
#Q38
#poets born in germany
queryString = """
SELECT (COUNT(DISTINCT ?s) AS ?Poets)
WHERE { 

?s  wdt:P106 wd:Q49757; #poet
    wdt:P19 ?birth.
?birth wdt:P17 wd:Q183. 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('Poets', '2050')]


1

Retrieve poets born,death o buried in France or Germany.

In [29]:
#Q39
#try to use the information found in the previous task
#poets born,death o buried in France or Germany
queryString = """
SELECT (COUNT(DISTINCT ?s) AS ?Born) (COUNT(DISTINCT ?d) AS ?Dead) (COUNT(DISTINCT ?b) AS ?Buried)
WHERE { 

{?s  wdt:P106 wd:Q49757; #poet
    wdt:P19 ?birth.
?birth wdt:P17 ?country.}

UNION{
  ?d  wdt:P106 wd:Q49757; #poet
      wdt:P20 ?death.
?death wdt:P17 ?country.}

UNION{
  ?b  wdt:P106 wd:Q49757; #poet
      wdt:P119 ?burial.
?burial wdt:P17 ?country.}

FILTER( (?country=wd:Q142) || (?country=wd:Q183) )

} 
"""

print("Results")
run_query(queryString)

Results
[('Born', '5892'), ('Dead', '4530'), ('Buried', '959')]


1

As above but consider also writers.

In [2]:
#Q40
#add writers to previous query
queryString = """
SELECT (COUNT(DISTINCT ?s) AS ?Born) (COUNT(DISTINCT ?d) AS ?Dead) (COUNT(DISTINCT ?b) AS ?Buried)
WHERE { 

{?s  wdt:P106 ?job; #poet
    wdt:P19 ?birth.
?birth wdt:P17 ?country.}

UNION{
  ?d  wdt:P106 ?job; #poet
      wdt:P20 ?death.
?death wdt:P17 ?country.}

UNION{
  ?b  wdt:P106 ?job; #poet
      wdt:P119 ?burial.
?burial wdt:P17 ?country.}

FILTER( ( (?country=wd:Q142) || (?country=wd:Q183) ) && ( (?job=wd:Q49757) || (?job=wd:Q36180) ) )

} 
"""

print("Results")
run_query(queryString)

Results
[('Born', '37110'), ('Dead', '21573'), ('Buried', '2956')]


1

Compare writers born in France and in Germany.

In [3]:
#Q41
#comparison between writers born in France and in Germany
queryString = """
SELECT ?french ?german 
WHERE{
{
SELECT (COUNT(DISTINCT(?f)) AS ?french) WHERE{
?f wdt:P106 wd:Q36180;
   wdt:P19 ?france.
?france wdt:P17 wd:Q142.
}}
{
SELECT (COUNT(DISTINCT(?g)) AS ?german) WHERE{
?g wdt:P106 wd:Q36180;
   wdt:P19 ?germany.
?germany wdt:P17 wd:Q183.
}}
}
"""

print("Results")
run_query(queryString)

Results
[('french', '11165'), ('german', '23065')]


1

Retrieve poets born and dead in france but not buried.

In [5]:
#Q42
#look for poets born and dead in france but not buried
queryString = """
SELECT DISTINCT ?s ?sname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19/wdt:P17 wd:Q142; #born in france
   wdt:P20/wdt:P17 wd:Q142; #dead in france
   wdt:P119/wdt:P17 ?country.

FILTER(?country!=wd:Q142)

#Labels
?s <http://schema.org/name> ?sname .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q44361'), ('sname', 'Ludwig I of Bavaria')]
[('s', 'http://www.wikidata.org/entity/Q184622'), ('sname', 'Jean Genet')]
[('s', 'http://www.wikidata.org/entity/Q170353'), ('sname', 'René of Anjou')]
[('s', 'http://www.wikidata.org/entity/Q272'), ('sname', 'Paul Morand')]
[('s', 'http://www.wikidata.org/entity/Q245355'), ('sname', 'Zygmunt Krasiński')]


5

As above but for German ones.

In [6]:
#Q43
#same but for german
queryString = """
SELECT DISTINCT ?s ?sname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19/wdt:P17 wd:Q183; #born in france
   wdt:P20/wdt:P17 wd:Q183; #dead in france
   wdt:P119/wdt:P17 ?country.

FILTER(?country!=wd:Q183)

#Labels
?s <http://schema.org/name> ?sname .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q57945'), ('sname', 'Georg Herwegh')]


1

Retrieve how many french poets died in Germany.

In [7]:
#Q44
#french poets who died in germany
queryString = """
SELECT (COUNT (DISTINCT ?s) AS ?French)
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19/wdt:P17 wd:Q142; #born in france
   wdt:P20/wdt:P17 wd:Q183. #dead in germany
}

"""

print("Results")
run_query(queryString)

Results
[('French', '26')]


1

In [8]:
#Q45
#previous one but reverse
#german poets who died in france
queryString = """
SELECT (COUNT (DISTINCT ?s) AS ?German)
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19/wdt:P17 wd:Q183; #born in ger
   wdt:P20/wdt:P17 wd:Q142. #dead in fra
}

"""

print("Results")
run_query(queryString)

Results
[('German', '30')]


1

<h3> TASK 3 ENDS HERE </h3>
Using the result of previous task i have been able to count how many poets and writers were born, or dead or buried in France or Germany (Q40). As a result I obtained ('Born', '37110'), ('Dead', '21573'), ('Buried', '2956').
I discovered Germany has more writers but less poets than France.
On top of that i tried to mix up place of birth, death and burial to see some results, for example 30 german poets died in France while 26
french poets died in Germany. I thought that could be interesting because many poets and writers had to fight in foreign countries during WWI and WWII.

<h3> TASK 4 STARTS HERE </h3>
4.1 Is there any poet for which the birth place and the place of burial are located in the same city either in Germany or France?
4.2 Which cities host the place of birth of the larger number of poets or writers across the two countries?
4.3 What are the top 3 cities in each country that you could visit? Based on what criteria?

Retrieve some properties of place of birth and burial.

In [11]:
#Q46
#poets with the same place of birth and burial
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9472'), ('pname', 'Cimiterium ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3008'), ('pname', 'Australian Heritage Database Place ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3449'), ('pname', 'NSW Heritage database ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2382'), ('pname', 'Chemins de mémoire ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4352'), ('pname', 'BillionGraves cemetery ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7673'), ('pname', 'Czech district ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7607'), ('pname', 'WikiTree category or space')]
[('p', 'http://www.wikidata.org/prop/direct/P8911'), ('pname', 'Sonneveld-index')]
[('p', 'http://www.wikidata.org/prop/direct/P1004'), ('pname', 'MusicBrainz place ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pname', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Class

25

Check for geographic property by adding a country in the path.

In [12]:
#Q47
#check for geographic property by adding a country in the path
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial ?p ?o.
?o wdt:P17 wd:Q142. 

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pname', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P1376'), ('pname', 'capital of')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pname', 'heritage designation')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P190'), ('pname', 'twinned administrative body')]
[('p', 'http://www.wikidata.org/prop/direct/P206'), ('pname', 'located in or next to body of water')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
[('p', 'http://www.wikidata.org

25

Use P131 to retrieve the geolocation of a certain place of birth/burial.

In [13]:
#Q48
#use P131 found above
queryString = """
SELECT DISTINCT ?bname ?o ?oname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P131 ?o;
        wdt:P17 wd:Q142.

#Labels
?burial <http://schema.org/name> ?bname .
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('bname', 'Cimetière Saint-Martin de Brest'), ('o', 'http://www.wikidata.org/entity/Q12193'), ('oname', 'Brest')]
[('bname', 'Véretz'), ('o', 'http://www.wikidata.org/entity/Q12556'), ('oname', 'Indre-et-Loire')]
[('bname', 'Véretz'), ('o', 'http://www.wikidata.org/entity/Q702657'), ('oname', 'arrondissement of Tours')]
[('bname', 'Trégastel'), ('o', 'http://www.wikidata.org/entity/Q3349'), ('oname', "Côtes-d'Armor")]
[('bname', 'Trégastel'), ('o', 'http://www.wikidata.org/entity/Q584641'), ('oname', 'arrondissement of Lannion')]
[('bname', 'Omonville-la-Petite'), ('o', 'http://www.wikidata.org/entity/Q700914'), ('oname', 'arrondissement of Cherbourg')]
[('bname', 'Omonville-la-Petite'), ('o', 'http://www.wikidata.org/entity/Q1724422'), ('oname', 'canton of Beaumont-Hague')]
[('bname', 'Omonville-la-Petite'), ('o', 'http://www.wikidata.org/entity/Q12589'), ('oname', 'Manche')]
[('bname', 'Saint-Germain-en-Coglès'), ('o', 'http://www.wikidata.org/entity/Q701222'), ('oname', 'ar

25

Use P669 (located on street) in order to get some info about location.

In [14]:
#Q49
#use P669 found on Q47
queryString = """
SELECT DISTINCT ?bname ?o ?oname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P669 ?o;
        wdt:P17 wd:Q142.

#Labels
?burial <http://schema.org/name> ?bname .
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('bname', 'cemetery of Fontainebleau'), ('o', 'http://www.wikidata.org/entity/Q101656142'), ('oname', 'Dead Fighters for France Square')]
[('bname', 'Church of St. Eustace'), ('o', 'http://www.wikidata.org/entity/Q3452393'), ('oname', 'rue du Jour')]
[('bname', 'Crématorium-columbarium du Père-Lachaise'), ('o', 'http://www.wikidata.org/entity/Q311'), ('oname', 'Père Lachaise Cemetery')]
[('bname', 'Cimetière de Bagneux'), ('o', 'http://www.wikidata.org/entity/Q104695867'), ('oname', 'Category:Avenue Marx-Dormoy (Bagneux et Montrouge)')]
[('bname', 'Cimetière de Pantin'), ('o', 'http://www.wikidata.org/entity/Q101210088'), ('oname', 'Avenue Jean-Jaurès')]
[('bname', 'Saint-Paul-Saint-Louis'), ('o', 'http://www.wikidata.org/entity/Q1478811'), ('oname', 'rue Saint-Antoine')]
[('bname', "Cimetière parisien d'Ivry"), ('o', 'http://www.wikidata.org/entity/Q99604296'), ('oname', 'Avenue de Verdun')]
[('bname', "Cimetière de l'Est"), ('o', 'http://www.wikidata.org/entity/Q20205330'), 

25

Use the graph pattern found above and retrieve some properties.

In [15]:
#Q49
#use P669 found on Q47
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P669 ?street;
        wdt:P17 wd:Q142.
?street ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1766'), ('pname', 'place name sign')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2046'), ('pname', 'area')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3182'), ('pname', 'FANTOIR code')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P2382'), ('pname', 'Chemins de mémoire ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4352'), ('pname', 'BillionGraves cemetery ID')]
[('p

25

Use the graph pattern of Q48 and also add country=France. Retrieve some properties.

In [17]:
#Q50
#use P669 found on Q47
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P669 ?street;
        wdt:P17 wd:Q142.
?street ?p ?o.
?o wdt:P17 wd:Q142.

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pname', 'heritage designation')]
[('p', 'http://www.wikidata.org/prop/direct/P47'), ('pname', 'shares border with')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P559'), ('pname', 'terminus')]
[('p', 'http://www.wikidata.org/prop/direct/P669'), ('pname', 'located on street')]
[('p', 'http://www.wikidata.org/prop/direct/P1427'), ('pname', 'start point')]
[('p', 'http://www.wikidata.org/prop/direct/P1444'), ('pname', 'destination point')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]


14

P706 is "located in", I think that it could be useful so I am going to use it in the next query, on top of the graph pattern used previously.

In [18]:
#Q51
#use P706 found above
queryString = """
SELECT DISTINCT ?o ?oname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P669 ?street;
        wdt:P17 wd:Q142.
?street wdt:P706 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q211034'), ('oname', 'île Saint-Louis')]
[('o', 'http://www.wikidata.org/entity/Q190063'), ('oname', 'île de la Cité')]


2

Run the same qiery as above but wihtout the "french" constraint in order to see if the pattern is valid in general.

In [19]:
#Q52
#use P706 found above
queryString = """
SELECT DISTINCT ?o ?oname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P706 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1815284'), ('oname', 'Leinegraben')]
[('o', 'http://www.wikidata.org/entity/Q10883263'), ('oname', 'Kyoto Basin')]
[('o', 'http://www.wikidata.org/entity/Q1769836'), ('oname', 'Veierland')]
[('o', 'http://www.wikidata.org/entity/Q2381124'), ('oname', 'South Lower Saxony')]
[('o', 'http://www.wikidata.org/entity/Q27600'), ('oname', 'Vasilyevsky Island')]
[('o', 'http://www.wikidata.org/entity/Q2641546'), ('oname', 'southern Ukraine')]
[('o', 'http://www.wikidata.org/entity/Q27449'), ('oname', 'Southern Europe')]
[('o', 'http://www.wikidata.org/entity/Q191862'), ('oname', 'East European Plain')]
[('o', 'http://www.wikidata.org/entity/Q14324291'), ('oname', 'South European Russia')]
[('o', 'http://www.wikidata.org/entity/Q15621281'), ('oname', 'Mont-Royal Heritage Site')]
[('o', 'http://www.wikidata.org/entity/Q72499'), ('oname', 'Mediterranean Basin')]
[('o', 'http://www.wikidata.org/entity/Q805369'), ('oname', 'metropolitan area Leipzig-Ha

25

Some results are pretty vague, so it's better to dig a little more on the properties to see if there is something useful.

In [20]:
#Q53
#closer look at wd:Q272208
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

wd:Q272208 ?p ?o.
?o wdt:P17 wd:Q142.

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P669'), ('pname', 'located on street')]
[('p', 'http://www.wikidata.org/prop/direct/P3032'), ('pname', 'adjacent building')]


7

In [21]:
#Q54
#closer look at wd:Q272208
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE{

wd:Q272208 ?p ?o.
?o wdt:P17 wd:Q142.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q187153'), ('oname', '14th arrondissement of Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q142'), ('oname', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q2972684'), ('oname', 'Parisian cemetery')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q142'), ('oname', 'France')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3220192'), ('oname', 'The Kiss')]
[('p', 'http://www.wikidata.org/prop/direct/P669'), ('pname', 'located on street'), ('o', 'http://www.wikidata.org/entity/Q2921810'), ('oname', 'boulevard Edgar-Quinet')]
[('p', 'http://w

7

In [22]:
#Q55
#wd:Q187153
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE{

wd:Q187153 ?p ?o.
?o wdt:P17 wd:Q142.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q90'), ('oname', 'Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pname', 'office held by head of government'), ('o', 'http://www.wikidata.org/entity/Q62093207'), ('oname', 'Mayor of 14th arrondissement of Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q234728'), ('oname', 'Vanves')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q640102'), ('oname', 'Gentilly')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q15149909'), ('oname', 'former 12th arrondissement of Paris')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q241021'), ('oname', 'M

17

Retrieve a burial place and the city where is located.

In [23]:
#Q56
#try to find city where burial place is located
queryString = """
SELECT DISTINCT ?burial ?bname ?city ?cname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P131* ?city.
?city wdt:P17 wd:Q142.

#Labels
?burial <http://schema.org/name> ?bname .
?city <http://schema.org/name> ?cname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('burial', 'http://www.wikidata.org/entity/Q317271'), ('bname', 'Verneuil-sur-Avre'), ('city', 'http://www.wikidata.org/entity/Q317271'), ('cname', 'Verneuil-sur-Avre')]
[('burial', 'http://www.wikidata.org/entity/Q317271'), ('bname', 'Verneuil-sur-Avre'), ('city', 'http://www.wikidata.org/entity/Q702864'), ('cname', 'arrondissement of Évreux')]
[('burial', 'http://www.wikidata.org/entity/Q317271'), ('bname', 'Verneuil-sur-Avre'), ('city', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France')]
[('burial', 'http://www.wikidata.org/entity/Q317271'), ('bname', 'Verneuil-sur-Avre'), ('city', 'http://www.wikidata.org/entity/Q212429'), ('cname', 'Metropolitan France')]
[('burial', 'http://www.wikidata.org/entity/Q317271'), ('bname', 'Verneuil-sur-Avre'), ('city', 'http://www.wikidata.org/entity/Q3372'), ('cname', 'Eure')]
[('burial', 'http://www.wikidata.org/entity/Q317271'), ('bname', 'Verneuil-sur-Avre'), ('city', 'http://www.wikidata.org/entity/Q18677875'), ('cname', 'Norma

25

Try to find a way to identify the category of "city".

In [24]:
#Q57
#there is a way to select only cities
queryString = """
SELECT DISTINCT ?o ?oname
WHERE{

wd:Q6602 wdt:P31 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1549591'), ('oname', 'big city')]
[('o', 'http://www.wikidata.org/entity/Q484170'), ('oname', 'commune of France')]
[('o', 'http://www.wikidata.org/entity/Q902814'), ('oname', 'border town')]


3

Retrieve the poets and the city where they are buried.

In [25]:
#Q58
#poets buried in a big city
queryString = """
SELECT DISTINCT ?burial ?bname ?city ?cname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P131* ?city.
?city wdt:P31 wd:Q1549591.

#Labels
?burial <http://schema.org/name> ?bname .
?city <http://schema.org/name> ?cname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('burial', 'http://www.wikidata.org/entity/Q19853853'), ('bname', 'the Tomb of Zheng Yong-xi'), ('city', 'http://www.wikidata.org/entity/Q249994'), ('cname', 'Hsinchu')]
[('burial', 'http://www.wikidata.org/entity/Q1358639'), ('bname', 'Westwood Village Memorial Park Cemetery'), ('city', 'http://www.wikidata.org/entity/Q65'), ('cname', 'Los Angeles')]
[('burial', 'http://www.wikidata.org/entity/Q168886'), ('bname', 'Powązki Cemetery'), ('city', 'http://www.wikidata.org/entity/Q270'), ('cname', 'Warsaw')]
[('burial', 'http://www.wikidata.org/entity/Q2790054'), ('bname', 'Powązki Military Cemetery'), ('city', 'http://www.wikidata.org/entity/Q270'), ('cname', 'Warsaw')]
[('burial', 'http://www.wikidata.org/entity/Q3033'), ('bname', 'Göttingen'), ('city', 'http://www.wikidata.org/entity/Q3033'), ('cname', 'Göttingen')]
[('burial', 'http://www.wikidata.org/entity/Q1627748'), ('bname', 'Hoppenlau Cemetery'), ('city', 'http://www.wikidata.org/entity/Q1022'), ('cname', 'Stuttgart')]
[

25

I am trying to find a more general definition of city using the entity "big city" found previously.

In [26]:
#Q59
#wd:Q1549591
queryString = """
SELECT DISTINCT ?o ?oname
WHERE{

wd:Q1549591 wdt:P31 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [27]:
#Q60
#wd:Q1549591
queryString = """
SELECT DISTINCT ?o ?oname
WHERE{

wd:Q1549591 ?p ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1637706'), ('oname', 'million city')]
[('o', 'http://www.wikidata.org/entity/Q3957'), ('oname', 'town')]
[('o', 'http://www.wikidata.org/entity/Q515'), ('oname', 'city')]
[('o', 'http://www.wikidata.org/entity/Q51706937'), ('oname', 'minimum population')]
[('o', 'http://www.wikidata.org/entity/Q9823231'), ('oname', 'Category:Big cities')]


5

In [28]:
#Q61
#wd:Q1549591 + city
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

wd:Q1549591 ?p wd:Q515.

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]


1

Retrieve poets and the cities where they are buried using the pattern found above.

In [29]:
#Q62
#poets buried in a city
queryString = """
SELECT DISTINCT ?burial ?bname ?city ?cname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P119 ?burial. #dead
?burial wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

#Labels
?burial <http://schema.org/name> ?bname .
?city <http://schema.org/name> ?cname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('burial', 'http://www.wikidata.org/entity/Q19853853'), ('bname', 'the Tomb of Zheng Yong-xi'), ('city', 'http://www.wikidata.org/entity/Q249994'), ('cname', 'Hsinchu')]
[('burial', 'http://www.wikidata.org/entity/Q722732'), ('bname', 'Litomyšl'), ('city', 'http://www.wikidata.org/entity/Q722732'), ('cname', 'Litomyšl')]
[('burial', 'http://www.wikidata.org/entity/Q1358639'), ('bname', 'Westwood Village Memorial Park Cemetery'), ('city', 'http://www.wikidata.org/entity/Q65'), ('cname', 'Los Angeles')]
[('burial', 'http://www.wikidata.org/entity/Q1508013'), ('bname', 'Jelgava Palace'), ('city', 'http://www.wikidata.org/entity/Q179830'), ('cname', 'Jelgava')]
[('burial', 'http://www.wikidata.org/entity/Q168886'), ('bname', 'Powązki Cemetery'), ('city', 'http://www.wikidata.org/entity/Q270'), ('cname', 'Warsaw')]
[('burial', 'http://www.wikidata.org/entity/Q2790054'), ('bname', 'Powązki Military Cemetery'), ('city', 'http://www.wikidata.org/entity/Q270'), ('cname', 'Warsaw')]
[('

25

In [30]:
#Q63
#closer look to place of birth
queryString = """
SELECT DISTINCT ?birth ?bname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth.
?birth wdt:P17 wd:Q142.

#Labels
?birth <http://schema.org/name> ?bname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('birth', 'http://www.wikidata.org/entity/Q752284'), ('bname', 'Joch')]
[('birth', 'http://www.wikidata.org/entity/Q152027'), ('bname', 'Saint-Nazaire')]
[('birth', 'http://www.wikidata.org/entity/Q15950541'), ('bname', "Hôtel d'Antoine")]
[('birth', 'http://www.wikidata.org/entity/Q1331813'), ('bname', 'Lanty-sur-Aube')]
[('birth', 'http://www.wikidata.org/entity/Q844680'), ('bname', 'Recey-sur-Ource')]
[('birth', 'http://www.wikidata.org/entity/Q638656'), ('bname', 'Vaucresson')]
[('birth', 'http://www.wikidata.org/entity/Q545015'), ('bname', 'Chaptuzat')]
[('birth', 'http://www.wikidata.org/entity/Q1163139'), ('bname', 'Le Soler')]
[('birth', 'http://www.wikidata.org/entity/Q737436'), ('bname', 'Chaumont-en-Vexin')]
[('birth', 'http://www.wikidata.org/entity/Q989360'), ('bname', 'Nieul-sur-Mer')]
[('birth', 'http://www.wikidata.org/entity/Q172455'), ('bname', 'Boulogne-Billancourt')]
[('birth', 'http://www.wikidata.org/entity/Q184159'), ('bname', 'Niort')]
[('birth', 'http:

25

I am checking if P19 has always a city as object because I noticed that some poets have as place of birth  arrondissement of Paris instead of Paris.

In [32]:
#Q64
#check if P19 retrieve a city or something else (for example above we have arrondissement of Paris instead of Paris)
queryString = """
SELECT DISTINCT ?birth ?bname ?city ?cname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth.
?birth wdt:P17 wd:Q142.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

#Labels
?birth <http://schema.org/name> ?bname .
?city <http://schema.org/name> ?cname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('birth', 'http://www.wikidata.org/entity/Q989360'), ('bname', 'Nieul-sur-Mer'), ('city', 'http://www.wikidata.org/entity/Q82185'), ('cname', 'La Rochelle')]
[('birth', 'http://www.wikidata.org/entity/Q172455'), ('bname', 'Boulogne-Billancourt'), ('city', 'http://www.wikidata.org/entity/Q172455'), ('cname', 'Boulogne-Billancourt')]
[('birth', 'http://www.wikidata.org/entity/Q41876'), ('bname', 'Reims'), ('city', 'http://www.wikidata.org/entity/Q41876'), ('cname', 'Reims')]
[('birth', 'http://www.wikidata.org/entity/Q42168'), ('bname', 'Clermont-Ferrand'), ('city', 'http://www.wikidata.org/entity/Q42168'), ('cname', 'Clermont-Ferrand')]
[('birth', 'http://www.wikidata.org/entity/Q12191'), ('bname', 'Nantes'), ('city', 'http://www.wikidata.org/entity/Q12191'), ('cname', 'Nantes')]
[('birth', 'http://www.wikidata.org/entity/Q15950541'), ('bname', "Hôtel d'Antoine"), ('city', 'http://www.wikidata.org/entity/Q47465'), ('cname', 'Aix-en-Provence')]
[('birth', 'http://www.wikidata.or

25

Retrieve german or french poets who were born and dead in the same city.

In [33]:
#TASK 4.1
#compare city of birth and dead in france and germany
queryString = """
SELECT DISTINCT ?sname ?bname ?cname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth;
   wdt:P119 ?burial.

?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

?burial wdt:P131* ?bcity.  #found location
?bcity wdt:P31 ?bo.
?bo wdt:P279 wd:Q515.

FILTER( ( (?country=wd:Q142)||(?country=wd:Q183) ) && (?city=?bcity) )
#Labels
?s <http://schema.org/name> ?sname .
?birth <http://schema.org/name> ?bname .
?city <http://schema.org/name> ?cname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('sname', 'Joseph Victor von Scheffel'), ('bname', 'Karlsruhe'), ('cname', 'Karlsruhe')]
[('sname', 'Johann Georg Daniel Arnold'), ('bname', 'Strasbourg'), ('cname', 'Strasbourg')]
[('sname', 'Ludwig Rubiner'), ('bname', 'Berlin'), ('cname', 'Berlin')]
[('sname', 'Hanns Heinz Ewers'), ('bname', 'Düsseldorf'), ('cname', 'Düsseldorf')]
[('sname', 'Otto Ernst'), ('bname', 'Ottensen'), ('cname', 'Hamburg')]
[('sname', 'Hans Sachs'), ('bname', 'Nuremberg'), ('cname', 'Nuremberg')]
[('sname', 'Roger Gilbert-Lecomte'), ('bname', 'Reims'), ('cname', 'Reims')]
[('sname', 'Sophie Hoechstetter'), ('bname', 'Pappenheim'), ('cname', 'Pappenheim')]
[('sname', 'Wilhelm Müller'), ('bname', 'Dessau'), ('cname', 'Dessau-Roßlau')]
[('sname', 'Wilhelm Hauff'), ('bname', 'Stuttgart'), ('cname', 'Stuttgart')]
[('sname', 'Édouard Turquety'), ('bname', 'Rennes'), ('cname', 'Rennes')]
[('sname', 'August Hermann Niemeyer'), ('bname', 'Halle (Saale)'), ('cname', 'Halle (Saale)')]
[('sname', 'Martin Heid

25

Count how many german or french poets were born and dead in the same city.

In [35]:
#Q65
#as above but with count
queryString = """
SELECT (COUNT (DISTINCT ?s) AS ?Poets)
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth;
   wdt:P119 ?burial.

?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

?burial wdt:P131* ?bcity.  #found location
?bcity wdt:P31 ?bo.
?bo wdt:P279 wd:Q515.

FILTER( ( (?country=wd:Q142)||(?country=wd:Q183) ) && (?city=?bcity) )
}
"""

print("Results")
run_query(queryString)

Results
[('Poets', '57')]


1

In [36]:
#Q66
#57 seems low, delete a constraint on birth
queryString = """
SELECT (COUNT (DISTINCT ?s) AS ?Poets)
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth;
   wdt:P119 ?burial.

?birth wdt:P17 ?country.

?burial wdt:P131* ?bcity.  #found location
?bcity wdt:P31 ?bo.
?bo wdt:P279 wd:Q515.

FILTER( ( (?country=wd:Q142)||(?country=wd:Q183) ) && (?birth=?bcity) )
}
"""

print("Results")
run_query(queryString)

Results
The operation failed timed out


<h3> TASK 4.1 ENDS HERE </h3>
I found the graph pattern to retrieve a city where a place of burial is located. This BGP made me able to see how many german and french poets were
born and dead in the same city. 

<h3> TASK 4.2 STARTS HERE </h3>
Which cities host the place of birth of the larger number of poets or writers across the two countries?

Retrieve german and french cities ordered by the number of poets that were born there.

In [4]:
#Q67
#try count on poets
queryString = """
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q64'), ('HowMany', '167')]
[('city', 'http://www.wikidata.org/entity/Q23482'), ('HowMany', '86')]
[('city', 'http://www.wikidata.org/entity/Q1055'), ('HowMany', '75')]
[('city', 'http://www.wikidata.org/entity/Q1726'), ('HowMany', '64')]
[('city', 'http://www.wikidata.org/entity/Q1479'), ('HowMany', '53')]
[('city', 'http://www.wikidata.org/entity/Q30974'), ('HowMany', '49')]
[('city', 'http://www.wikidata.org/entity/Q7880'), ('HowMany', '43')]
[('city', 'http://www.wikidata.org/entity/Q1022'), ('HowMany', '41')]
[('city', 'http://www.wikidata.org/entity/Q6602'), ('HowMany', '41')]
[('city', 'http://www.wikidata.org/entity/Q365'), ('HowMany', '40')]
[('city', 'http://www.wikidata.org/entity/Q1731'), ('HowMany', '38')]
[('city', 'http://www.wikidata.org/entity/Q1794'), ('HowMany', '38')]
[('city', 'http://www.wikidata.org/entity/Q42807'), ('HowMany', '34')]
[('city', 'http://www.wikidata.org/entity/Q2079'), ('HowMany', '33')]
[('city', '

25

Retrieve german and french cities ordered by the number of poets and writers that were born there.

In [5]:
#Q68
queryString = """
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P106 ?job; 
   wdt:P19 ?birth.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( ( (?country=wd:Q142)||(?country=wd:Q183) ) && ( (?job=wd:Q49757) || (?job=wd:Q36180) ) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q64'), ('HowMany', '2028')]
[('city', 'http://www.wikidata.org/entity/Q1055'), ('HowMany', '941')]
[('city', 'http://www.wikidata.org/entity/Q1726'), ('HowMany', '783')]
[('city', 'http://www.wikidata.org/entity/Q2079'), ('HowMany', '466')]
[('city', 'http://www.wikidata.org/entity/Q365'), ('HowMany', '466')]
[('city', 'http://www.wikidata.org/entity/Q1794'), ('HowMany', '432')]
[('city', 'http://www.wikidata.org/entity/Q1022'), ('HowMany', '396')]
[('city', 'http://www.wikidata.org/entity/Q1731'), ('HowMany', '386')]
[('city', 'http://www.wikidata.org/entity/Q1715'), ('HowMany', '314')]
[('city', 'http://www.wikidata.org/entity/Q1209'), ('HowMany', '280')]
[('city', 'http://www.wikidata.org/entity/Q23482'), ('HowMany', '262')]
[('city', 'http://www.wikidata.org/entity/Q1718'), ('HowMany', '261')]
[('city', 'http://www.wikidata.org/entity/Q2090'), ('HowMany', '233')]
[('city', 'http://www.wikidata.org/entity/Q2107'), ('HowMany', '194')]

25

In [3]:
#TASK 4.2:
queryString = """
SELECT ?city ?cname ?HowMany WHERE
{
{
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P106 ?job; 
   wdt:P19 ?birth.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( ( (?country=wd:Q142)||(?country=wd:Q183) ) && ( (?job=wd:Q49757) || (?job=wd:Q36180) ) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
}
#Labels
?city <http://schema.org/name> ?cname .
}
ORDER BY DESC(?HowMany)
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q64'), ('cname', 'Berlin'), ('HowMany', '2028')]
[('city', 'http://www.wikidata.org/entity/Q1055'), ('cname', 'Hamburg'), ('HowMany', '941')]
[('city', 'http://www.wikidata.org/entity/Q1726'), ('cname', 'Munich'), ('HowMany', '783')]
[('city', 'http://www.wikidata.org/entity/Q2079'), ('cname', 'Leipzig'), ('HowMany', '466')]
[('city', 'http://www.wikidata.org/entity/Q365'), ('cname', 'Cologne'), ('HowMany', '466')]
[('city', 'http://www.wikidata.org/entity/Q1794'), ('cname', 'Frankfurt am Main'), ('HowMany', '432')]
[('city', 'http://www.wikidata.org/entity/Q1022'), ('cname', 'Stuttgart'), ('HowMany', '396')]
[('city', 'http://www.wikidata.org/entity/Q1731'), ('cname', 'Dresden'), ('HowMany', '386')]
[('city', 'http://www.wikidata.org/entity/Q1715'), ('cname', 'Hanover'), ('HowMany', '314')]
[('city', 'http://www.wikidata.org/entity/Q1209'), ('cname', 'Free Hanseatic City of Bremen'), ('HowMany', '280')]
[('city', 'http://www.wikidata.o

25

<h3> TASK 4.2 ENDS HERE </h3>
In the previous query I orderder german or french cities by the number of poets and writers born there. Most of them are born in
german cities such as Berlin, Hamburg and Munich

<h3> TASK 4.3 STARTS HERE </h3>
4.3 What are the top 3 cities in each country that you could visit? Based on what criteria?

Let's have a closer look to poets related to Berlin

In [6]:
#Q69
#let's have a closer look to poets related to Berlin
queryString = """
SELECT DISTINCT ?s ?sname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER(?city=wd:Q64)
#Labels
?s <http://schema.org/name> ?sname .

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1874926'), ('sname', 'Ludwig Rubiner')]
[('s', 'http://www.wikidata.org/entity/Q72568'), ('sname', 'Alfred Lichtenstein')]
[('s', 'http://www.wikidata.org/entity/Q57386'), ('sname', 'Kurt Tucholsky')]
[('s', 'http://www.wikidata.org/entity/Q88302'), ('sname', 'Karin Kiwus')]
[('s', 'http://www.wikidata.org/entity/Q90577'), ('sname', 'Christa Reinig')]
[('s', 'http://www.wikidata.org/entity/Q67815'), ('sname', 'Helmina von Chézy')]
[('s', 'http://www.wikidata.org/entity/Q110163'), ('sname', 'Carl Blum')]
[('s', 'http://www.wikidata.org/entity/Q70435'), ('sname', 'Ludwig Robert')]
[('s', 'http://www.wikidata.org/entity/Q76487'), ('sname', 'Paul Heyse')]
[('s', 'http://www.wikidata.org/entity/Q76664'), ('sname', 'Jakob van Hoddis')]
[('s', 'http://www.wikidata.org/entity/Q77933'), ('sname', 'Caroline von Briest')]
[('s', 'http://www.wikidata.org/entity/Q85155'), ('sname', 'Ferdinand Avenarius')]
[('s', 'http://www.wikidata.org/entity/Q103211

25

In [9]:
#Q70
#let's have a closer look to poets related to Berlin
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth;
   ?p ?obj.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER(?city=wd:Q64)

#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7947'), ('pname', 'WeChangEd ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9129'), ('pname', 'Data Collection of the Hungarian Peerage ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pname', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pname', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://w

25

Previous results are too generic, i want to find something related to their poetry. 

In [10]:
#Q71
#previous results are too generic, i want to find something related to their poetry
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth;
   ?p ?obj.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER(?city=wd:Q64)

#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7947'), ('pname', 'WeChangEd ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9129'), ('pname', 'Data Collection of the Hungarian Peerage ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pname', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pname', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://w

50

I found P135 (movement) and P136 (genre) that could be useful in the search.

In [12]:
#Q72
#movement (P135) and genre(P136) are interesting
queryString = """
SELECT DISTINCT ?genre ?gname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth;
   wdt:P135 ?genre.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER(?city=wd:Q64)

#Labels
?genre <http://schema.org/name> ?gname .

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q37068'), ('gname', 'Romanticism')]
[('genre', 'http://www.wikidata.org/entity/Q181655'), ('gname', 'Heidelberg School')]
[('genre', 'http://www.wikidata.org/entity/Q2477112'), ('gname', 'German Romanticism')]
[('genre', 'http://www.wikidata.org/entity/Q5977111'), ('gname', 'romantic literature')]
[('genre', 'http://www.wikidata.org/entity/Q177725'), ('gname', 'abstract expressionism')]


5

In [13]:
#Q73
#movement (P135) and genre(P136) are interesting
queryString = """
SELECT DISTINCT ?genre ?gname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth;
   wdt:P136 ?genre.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER(?city=wd:Q64)

#Labels
?genre <http://schema.org/name> ?gname .

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q1344'), ('gname', 'opera')]
[('genre', 'http://www.wikidata.org/entity/Q482'), ('gname', 'poetry')]
[('genre', 'http://www.wikidata.org/entity/Q7362788'), ('gname', 'Romantic')]
[('genre', 'http://www.wikidata.org/entity/Q37073'), ('gname', 'pop music')]


4

Retrieve some movements related to poets born in Berlin.

In [15]:
#Q74
#search for movement with the berlin pattern
queryString = """
SELECT DISTINCT ?genre ?gname
WHERE{

?s wdt:P106 wd:Q49757; #poet
   wdt:P19 ?birth;
   wdt:P135 ?genre.

?birth wdt:P17 ?country.

#Labels
?genre <http://schema.org/name> ?gname .

FILTER((?country=wd:Q142)||(?country=wd:Q183))
}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q207741'), ('gname', 'Sturm und Drang')]
[('genre', 'http://www.wikidata.org/entity/Q39427'), ('gname', 'surrealism')]
[('genre', 'http://www.wikidata.org/entity/Q40415'), ('gname', 'Impressionism')]
[('genre', 'http://www.wikidata.org/entity/Q42934'), ('gname', 'Cubism')]
[('genre', 'http://www.wikidata.org/entity/Q2477112'), ('gname', 'German Romanticism')]
[('genre', 'http://www.wikidata.org/entity/Q12539'), ('gname', 'Age of Enlightenment')]
[('genre', 'http://www.wikidata.org/entity/Q335652'), ('gname', 'parnassianism')]
[('genre', 'http://www.wikidata.org/entity/Q163775'), ('gname', 'medieval music')]
[('genre', 'http://www.wikidata.org/entity/Q37068'), ('gname', 'Romanticism')]
[('genre', 'http://www.wikidata.org/entity/Q37853'), ('gname', 'Baroque')]
[('genre', 'http://www.wikidata.org/entity/Q9730'), ('gname', 'Western classical music')]
[('genre', 'http://www.wikidata.org/entity/Q170292'), ('gname', 'classicism')]
[('genre', 

50

In [16]:
#Q75
#closer look to an entity found above
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

wd:Q5977111 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P349'), ('pname', 'National Diet Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('pname', 'Biblioteca Nacional de España ID')]


6

In [17]:
#Q76
#closer look to an entity found above
queryString = """
SELECT DISTINCT ?o ?oname
WHERE{

wd:Q5977111 wdt:P279 ?o.

#Labels
?o <http://schema.org/name> ?oname .

}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q37068'), ('oname', 'Romanticism')]
[('o', 'http://www.wikidata.org/entity/Q8242'), ('oname', 'literature')]


2

In [18]:
#Q77
#closer look to an entity found above
queryString = """
SELECT DISTINCT ?p ?pname
WHERE{

wd:Q37068 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pname', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pname', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pname', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1807'), ('pname', 'Great Aragonese Encyclopedia ID')]
[('p', 'http://www.wiki

50

Check if a poet can be directly linked to Romanticism by P135

In [21]:
#Q77
#check if a poet can be directly linked to Romanticism by P135
queryString = """
SELECT DISTINCT ?s ?sname
WHERE{

?s wdt:P106 wd:Q49757;
   wdt:P135 wd:Q37068.

#Labels
?s <http://schema.org/name> ?sname .

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5932129'), ('sname', 'Joaquín José Cervino')]
[('s', 'http://www.wikidata.org/entity/Q5258787'), ('sname', 'Tobias Barreto')]
[('s', 'http://www.wikidata.org/entity/Q5340521'), ('sname', 'Eduardo Asquerino')]
[('s', 'http://www.wikidata.org/entity/Q573223'), ('sname', 'Patricio de la Escosura')]
[('s', 'http://www.wikidata.org/entity/Q688529'), ('sname', 'Johannes Carsten Hauch')]
[('s', 'http://www.wikidata.org/entity/Q597298'), ('sname', 'Antoni Edward Odyniec')]
[('s', 'http://www.wikidata.org/entity/Q645491'), ('sname', 'Yevhen Hrebinka')]
[('s', 'http://www.wikidata.org/entity/Q93343'), ('sname', 'Percy Bysshe Shelley')]
[('s', 'http://www.wikidata.org/entity/Q378133'), ('sname', 'Hendrik Conscience')]
[('s', 'http://www.wikidata.org/entity/Q352914'), ('sname', 'Karel Jaromír Erben')]
[('s', 'http://www.wikidata.org/entity/Q2621545'), ('sname', 'Alexander Polezhayev')]
[('s', 'http://www.wikidata.org/entity/Q4151323'), ('sname', 'Edu

25

Retrieve subclasseses of Romanticism.

In [20]:
#Q78
#check P527 on Romanticism
queryString = """
SELECT DISTINCT ?o ?oname
WHERE{

wd:Q37068 wdt:P527 ?o.

#Labels
?o <http://schema.org/name> ?oname .

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3038946'), ('oname', 'romantic drama')]
[('o', 'http://www.wikidata.org/entity/Q207591'), ('oname', 'Romantic music')]
[('o', 'http://www.wikidata.org/entity/Q5977111'), ('oname', 'romantic literature')]
[('o', 'http://www.wikidata.org/entity/Q3088366'), ('oname', 'Romantic painting')]
[('o', 'http://www.wikidata.org/entity/Q12775412'), ('oname', 'Romantic philosophy')]


5

Retrieve the three cities where the highest number of romantic poets used to live (born).

In [22]:
#Q79
#find the three cities where the highest number of romantic poets used to live (born)
queryString = """
SELECT ?city ?cname ?HowMany WHERE
{
{
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P106 wd:Q49757; 
   wdt:P19 ?birth;
   wdt:P135 wd:Q37068.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
}
#Labels
?city <http://schema.org/name> ?cname .
}
ORDER BY DESC(?HowMany)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q1715'), ('cname', 'Hanover'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q37776'), ('cname', 'Besançon'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q64'), ('cname', 'Berlin'), ('HowMany', '2')]


3

As above but using romantic literature instead of romanticism.

In [26]:
#Q80
#find the three cities where the highest number of romantic poets used to live (born)
#use romantic literature instead of romanticism
queryString = """
SELECT ?city ?cname ?HowMany WHERE
{
{
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P106 wd:Q49757; 
   wdt:P19 ?birth;
   wdt:P135 wd:Q5977111.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
}
#Labels
?city <http://schema.org/name> ?cname .
}
ORDER BY DESC(?HowMany)
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q697577'), ('cname', 'Arnstein'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q1715'), ('cname', 'Hanover'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q64'), ('cname', 'Berlin'), ('HowMany', '1')]


3

In [27]:
#Q81
#use only country pattern
queryString = """
SELECT (COUNT(DISTINCT(?s)) AS ?RPoets) WHERE
{
?s wdt:P106 wd:Q49757; 
   wdt:P27 ?country;
   wdt:P135 wd:Q5977111.
   
FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
"""

print("Results")
run_query(queryString)

Results
[('RPoets', '3')]


1

Retrieve how many romantic poets are born in France or Germany.

In [28]:
#Q82
#use only country pattern + romanticism
queryString = """
SELECT (COUNT(DISTINCT(?s)) AS ?RPoets) WHERE
{
?s wdt:P106 wd:Q49757; 
   wdt:P27 ?country;
   wdt:P135 wd:Q37068.
   
FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
"""

print("Results")
run_query(queryString)

Results
[('RPoets', '23')]


1

Retrieve the most common movements in France and Germany.

In [30]:
#Q83
#search for most common movement in french or german poets
queryString = """
SELECT ?genre (COUNT(DISTINCT(?s)) AS ?Poets) WHERE
{
?s wdt:P106 wd:Q49757; 
   wdt:P27 ?country;
   wdt:P135 ?genre.
   
FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?genre)
ORDER BY DESC(?Poets)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q163775'), ('Poets', '34')]
[('genre', 'http://www.wikidata.org/entity/Q9730'), ('Poets', '30')]
[('genre', 'http://www.wikidata.org/entity/Q39427'), ('Poets', '30')]
[('genre', 'http://www.wikidata.org/entity/Q37068'), ('Poets', '23')]
[('genre', 'http://www.wikidata.org/entity/Q164800'), ('Poets', '19')]
[('genre', 'http://www.wikidata.org/entity/Q471194'), ('Poets', '19')]
[('genre', 'http://www.wikidata.org/entity/Q6034'), ('Poets', '17')]
[('genre', 'http://www.wikidata.org/entity/Q876031'), ('Poets', '12')]
[('genre', 'http://www.wikidata.org/entity/Q528648'), ('Poets', '11')]
[('genre', 'http://www.wikidata.org/entity/Q2455000'), ('Poets', '10')]


10

In [31]:
#Q84
#get labels
queryString = """
SELECT ?genre ?gname ?Poets WHERE{
{
SELECT ?genre (COUNT(DISTINCT(?s)) AS ?Poets) WHERE
{
?s wdt:P106 wd:Q49757; 
   wdt:P27 ?country;
   wdt:P135 ?genre.
   
FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?genre)
ORDER BY DESC(?Poets)
LIMIT 10
}
#Labels
?genre <http://schema.org/name> ?gname .
}
ORDER BY DESC(?Poets)
"""

print("Results")
run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q163775'), ('gname', 'medieval music'), ('Poets', '34')]
[('genre', 'http://www.wikidata.org/entity/Q39427'), ('gname', 'surrealism'), ('Poets', '30')]
[('genre', 'http://www.wikidata.org/entity/Q9730'), ('gname', 'Western classical music'), ('Poets', '30')]
[('genre', 'http://www.wikidata.org/entity/Q37068'), ('gname', 'Romanticism'), ('Poets', '23')]
[('genre', 'http://www.wikidata.org/entity/Q471194'), ('gname', 'troubadoric poetry'), ('Poets', '19')]
[('genre', 'http://www.wikidata.org/entity/Q164800'), ('gname', 'Symbolism'), ('Poets', '19')]
[('genre', 'http://www.wikidata.org/entity/Q6034'), ('gname', 'Dada'), ('Poets', '17')]
[('genre', 'http://www.wikidata.org/entity/Q876031'), ('gname', 'School of Lyon'), ('Poets', '12')]
[('genre', 'http://www.wikidata.org/entity/Q528648'), ('gname', 'La Pléiade'), ('Poets', '11')]
[('genre', 'http://www.wikidata.org/entity/Q2455000'), ('gname', 'German Renaissance'), ('Poets', '10')]


10

Retrieve some properties of romantic poets.

In [33]:
#Q85
#get some info about german romantic poets
queryString = """
SELECT DISTINCT ?p ?pname WHERE{

?s wdt:P106 wd:Q49757; 
   wdt:P27 wd:Q183;
   wdt:P135 wd:Q37068;
   ?p ?o.
#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 50

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7947'), ('pname', 'WeChangEd ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9129'), ('pname', 'Data Collection of the Hungarian Peerage ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('pname', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pname', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pname', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('pname', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p

50

In [35]:
#Q86
#check awards for german poets
queryString = """
SELECT DISTINCT ?award ?aname WHERE{

?s wdt:P106 wd:Q49757; 
   wdt:P27 wd:Q183;
   wdt:P166 ?award.
#Labels
?award <http://schema.org/name> ?aname .
}
LIMIT 50

"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1260597'), ('aname', 'Droste-Preis')]
[('award', 'http://www.wikidata.org/entity/Q1396769'), ('aname', 'Goethe-Medaille für Kunst und Wissenschaft')]
[('award', 'http://www.wikidata.org/entity/Q15124418'), ('aname', 'honorary citizen of Munich')]
[('award', 'http://www.wikidata.org/entity/Q1534190'), ('aname', 'Goethe Plaque of the City of Frankfurt')]
[('award', 'http://www.wikidata.org/entity/Q1595389'), ('aname', 'Thüringer Literaturpreis')]
[('award', 'http://www.wikidata.org/entity/Q17484040'), ('aname', 'Order of the Red Eagle 3rd Class')]
[('award', 'http://www.wikidata.org/entity/Q18582827'), ('aname', 'Order of the Red Eagle 4th Class')]
[('award', 'http://www.wikidata.org/entity/Q519214'), ('aname', 'Nelly Sachs Prize')]
[('award', 'http://www.wikidata.org/entity/Q700236'), ('aname', 'German Book Prize')]
[('award', 'http://www.wikidata.org/entity/Q43101241'), ('aname', 'Christine Lavant prize')]
[('award', 'http://www.wikid

50

Retrieve how many german or french poets and writers won the nobel prize.

In [36]:
#Q87
#check how many german or french poets/writers won the nobel
queryString = """
SELECT ?country (COUNT (DISTINCT ?s) AS ?HowMany) WHERE{

?s wdt:P106 ?job; 
   wdt:P27 ?country;
   wdt:P166 wd:Q37922.
   
FILTER( ( (?country=wd:Q142)||(?country=wd:Q183) ) && ( (?job=wd:Q49757) || (?job=wd:Q36180) ) )
}
GROUP BY(?country)
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('HowMany', '16')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('HowMany', '6')]


2

Retrieve top three cities for nobel prizes (poets) born there.

In [37]:
#Q88
#retrieve top three cities for nobel prizes (poets) born there
queryString = """
SELECT ?city ?cname ?HowMany WHERE
{
{
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P106 wd:Q49757; 
   wdt:P19 ?birth;
   wdt:P166 wd:Q37922.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
}
#Labels
?city <http://schema.org/name> ?cname .
}
ORDER BY DESC(?HowMany)
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q64'), ('cname', 'Berlin'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q365'), ('cname', 'Cologne'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q487921'), ('cname', 'Calw'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q1479'), ('cname', 'Bordeaux'), ('HowMany', '1')]


4

In [38]:
#Q89
#retrieve top three cities for nobel prizes (poets and writers) born there
queryString = """
SELECT ?city ?cname ?HowMany WHERE
{
{
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P106 ?job; 
   wdt:P19 ?birth;
   wdt:P166 wd:Q37922.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( ( (?country=wd:Q142)||(?country=wd:Q183) ) && ( (?job=wd:Q49757) || (?job=wd:Q36180) ) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
}
#Labels
?city <http://schema.org/name> ?cname .
}
ORDER BY DESC(?HowMany)
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q64'), ('cname', 'Berlin'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q14839'), ('cname', 'Aurich'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q172455'), ('cname', 'Boulogne-Billancourt'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q27324'), ('cname', 'Garding'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q2843'), ('cname', 'Lübeck'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q33959'), ('cname', 'Nice'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q365'), ('cname', 'Cologne'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q487921'), ('cname', 'Calw'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q1479'), ('cname', 'Bordeaux'), ('HowMany', '1')]


9

Retrieve top three cities for romantic artists born there.

In [39]:
#Q89
#retrieve top three cities for romantic artists born there
queryString = """
SELECT ?city ?cname ?HowMany WHERE
{
{
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P19 ?birth;
   wdt:P135 wd:Q37068.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
}
#Labels
?city <http://schema.org/name> ?cname .
}
ORDER BY DESC(?HowMany)
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q37776'), ('cname', 'Besançon'), ('HowMany', '4')]
[('city', 'http://www.wikidata.org/entity/Q1715'), ('cname', 'Hanover'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q365'), ('cname', 'Cologne'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q3802'), ('cname', 'Hanau'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q64'), ('cname', 'Berlin'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q525720'), ('cname', 'Lauffen am Neckar'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q3931'), ('cname', 'Brandenburg an der Havel'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q697577'), ('cname', 'Arnstein'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q1055'), ('cname', 'Hamburg'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q1711'), ('cname', 'Potsdam'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q1718'), ('c

22

Retrieve top three cities for neoclassic artists born there.

In [40]:
#Q90
#retrieve top three cities for neoclassic artists born there
queryString = """
SELECT ?city ?cname ?HowMany WHERE
{
{
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P19 ?birth;
   wdt:P135 wd:Q14378.
   
?birth wdt:P17 ?country.

?birth wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
}
#Labels
?city <http://schema.org/name> ?cname .
}
ORDER BY DESC(?HowMany)
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q64'), ('cname', 'Berlin'), ('HowMany', '4')]
[('city', 'http://www.wikidata.org/entity/Q586'), ('cname', 'Bonn'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q648'), ('cname', 'Lille'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q7880'), ('cname', 'Toulouse'), ('HowMany', '2')]
[('city', 'http://www.wikidata.org/entity/Q3905'), ('cname', 'Gladbeck'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q7050'), ('cname', 'Weinheim'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q498078'), ('cname', 'Bad Arolsen'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q12191'), ('cname', 'Nantes'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q2843'), ('cname', 'Lübeck'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q30974'), ('cname', 'Rouen'), ('HowMany', '1')]
[('city', 'http://www.wikidata.org/entity/Q37776'), ('cname', 'Besançon'), ('HowMany'

17

Retrieve top three cities with the highest number of poets buried

In [41]:
#Q91
#retrieve top three cities with the highest number of poets buried
queryString = """
SELECT ?city ?cname ?HowMany WHERE
{
{
SELECT ?city (COUNT (DISTINCT ?s) AS ?HowMany)
WHERE{

?s wdt:P119 ?burial;
   wdt:P106 wd:Q49757.
   
?burial wdt:P17 ?country.

?burial wdt:P131* ?city.  #found location
?city wdt:P31 ?o.
?o wdt:P279 wd:Q515.

FILTER( (?country=wd:Q142)||(?country=wd:Q183) )
}
GROUP BY (?city)
ORDER BY DESC(?HowMany)
LIMIT 25
}
#Labels
?city <http://schema.org/name> ?cname .
}
ORDER BY DESC(?HowMany)
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q64'), ('cname', 'Berlin'), ('HowMany', '51')]
[('city', 'http://www.wikidata.org/entity/Q1726'), ('cname', 'Munich'), ('HowMany', '23')]
[('city', 'http://www.wikidata.org/entity/Q1022'), ('cname', 'Stuttgart'), ('HowMany', '16')]
[('city', 'http://www.wikidata.org/entity/Q1055'), ('cname', 'Hamburg'), ('HowMany', '14')]
[('city', 'http://www.wikidata.org/entity/Q1794'), ('cname', 'Frankfurt am Main'), ('HowMany', '11')]
[('city', 'http://www.wikidata.org/entity/Q1731'), ('cname', 'Dresden'), ('HowMany', '10')]
[('city', 'http://www.wikidata.org/entity/Q3955'), ('cname', 'Weimar'), ('HowMany', '8')]
[('city', 'http://www.wikidata.org/entity/Q1715'), ('cname', 'Hanover'), ('HowMany', '7')]
[('city', 'http://www.wikidata.org/entity/Q23482'), ('cname', 'Marseille'), ('HowMany', '5')]
[('city', 'http://www.wikidata.org/entity/Q33959'), ('cname', 'Nice'), ('HowMany', '5')]
[('city', 'http://www.wikidata.org/entity/Q365'), ('cname', 'Cologne

25

<h3> TASK 4.3 ENDS HERE </h3>
I think i'd visit Berlin because leads for nobel prize for literature and romantic artists born there, Besancon which instead is first for romantic
artist born, and Munich, which is the second "cemetry" of german poets.